In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20660648
paper_name = 'pan_boeke_2010' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [12]:
files = ['TableS1.txt','TableS2.txt','TableS3.txt']
scores = [-2, -1, 1]

In [13]:
original_data_list = []
for ixf, f in enumerate(files):
    original_data = pd.read_csv('raw_data/' + f, sep='\n', header=None)
    print('Original data dimensions: %d x %d' % (original_data.shape))
#     print(original_data.head())
    
    original_data['orf'] = original_data[0].apply(lambda x: x.split(' ')[0])
    original_data['orf'] = original_data['orf'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data['data'] = scores[ixf]
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 33 x 1
Empty DataFrame
Columns: [0, orf]
Index: []
(33, 1)
Original data dimensions: 191 x 1
Empty DataFrame
Columns: [0, orf]
Index: []
(191, 1)
Original data dimensions: 109 x 1
Empty DataFrame
Columns: [0, orf]
Index: []
(109, 1)


In [14]:
original_data = pd.concat(original_data_list, axis=0)

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.head()

,data
orf,
YAL012W,-1.0
YAL024C,-1.0
YAL038W,-2.0
YAL055W,-1.0
YAL056W,-1.0


In [19]:
original_data.shape

(327, 1)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [109]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,109
data_type,value
orf,
YAL012W,-1.0
YAL024C,-1.0
YAL038W,-2.0
YAL055W,-1.0
YAL056W,-1.0


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,109
,data_type,value
gene_id,orf,
10,YAL012W,-1.0
22,YAL024C,-1.0
36,YAL038W,-2.0
51,YAL055W,-1.0
52,YAL056W,-1.0


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        109         
data_type       value   valuez
gene_id orf                   
10      YAL012W  -1.0 -3.31862
22      YAL024C  -1.0 -3.31862
36      YAL038W  -2.0 -6.63724
51      YAL055W  -1.0 -3.31862
52      YAL056W  -1.0 -3.31862

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 20660648...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.34it/s]

Updating the data_modified_on field...
